# Hyperparameter Tuning Methods Comparison
Cross validation approach based on the following [repo](https://github.com/roamanalytics/roamresearch/tree/master/BlogPosts/Hyperparameter_tuning_comparison)

In [196]:
%matplotlib inline

In [215]:
import dataset_loader as datasets
from hpt_cmp import *

In [216]:
%load_ext autoreload
%aimport hpt_cmp
%aimport dataset_loader
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [217]:
from __future__ import print_function
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

In [218]:
# Loading the Digits dataset
mth = 'load_'+'digits'
digits = datasets.load_digits()
iris = datasets.load_iris()
wine = datasets.load_wine()
bc = datasets.load_breast_cancer()
cs = datasets.load_census_50k()

all_datasets = [cs]
# To apply an classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:

In [219]:
param_grid = {
    'C': [0.1, 1, 10, 100, 1000],  
    'gamma': [0.0001, 0.001, 0.01, 0.1, 1.0],
    'kernel': ['linear', 'rbf']
}

bayes_grid ={
     'C': (0.1, 100, 'log-uniform'),
    'gamma': (0.0001, 1, 'log-uniform'),
        'kernel': ['linear', 'rbf']
}

hyperopt_grid = {
    'C': hp.choice('C', [0.1, 1, 10, 100, 1000]),
    'gamma': hp.choice('gamma',[0.0001, 0.001, 0.01, 0.1, 1.0]),
    'kernel': hp.choice('kernel', ['linear', 'rbf'])
}

In [220]:
hpt_objs = [
    # add more objs once more search functions implemented
        {
        'name': 'Baseline',
        'cv': run_baseline,
        'param_grid' : {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'},
        # {}, # use default config
        'args': {}
    },
    {
        'name': 'BaysianSearch',
        'cv' : baysian_search,
        'param_grid': bayes_grid,
        'args': {'n_iter':10}
    },
    {
        'name': 'Tree of Parzen Estimator',
        'cv' : tpe_search,
        'param_grid': hyperopt_grid,
        'args': {}
    },
    {
        'name': 'RandomSearch',
        'cv' : random_search,
        'param_grid': param_grid,
        'args': {'n_iter':10}
    },
    {
        'name': 'GridSearch',
        'cv': grid_search,
        'param_grid': param_grid,
        'args' : {}
    }
]

hpt_objs = [DotMap(obj, _dynamic=False) for obj in hpt_objs ]
hpt_objs

[DotMap(name='Baseline', cv=<function run_baseline at 0x7fc7cfd59400>, param_grid=DotMap(C=1, gamma=0.0001, kernel='rbf'), args=DotMap()),
 DotMap(name='BaysianSearch', cv=<function baysian_search at 0x7fc7c5ab6598>, param_grid=DotMap(C=(0.1, 100, 'log-uniform'), gamma=(0.0001, 1, 'log-uniform'), kernel=['linear', 'rbf']), args=DotMap(n_iter=10)),
 DotMap(name='Tree of Parzen Estimator', cv=<function tpe_search at 0x7fc7c5ab6378>, param_grid=DotMap(C=<hyperopt.pyll.base.Apply object at 0x7fc7c589d908>, gamma=<hyperopt.pyll.base.Apply object at 0x7fc7c589d9b0>, kernel=<hyperopt.pyll.base.Apply object at 0x7fc7c589d358>), args=DotMap()),
 DotMap(name='RandomSearch', cv=<function random_search at 0x7fc7c5ab68c8>, param_grid=DotMap(C=[0.1, 1, 10, 100, 1000], gamma=[0.0001, 0.001, 0.01, 0.1, 1.0], kernel=['linear', 'rbf']), args=DotMap(n_iter=10)),
 DotMap(name='GridSearch', cv=<function grid_search at 0x7fc7c5ab6730>, param_grid=DotMap(C=[0.1, 1, 10, 100, 1000], gamma=[0.0001, 0.001, 0.01,

#### cmp_hpt_methods
Parameter description
> `htp_objs`: list of hyperparam-tuning object <br>
> `model`: sklearn model to optimize (needs to have fit/predict function)<br>
> `dataset`: tuple of (X,y) e.g (Data, Target) <br>
> `loss`: sklearn loss function to user <br>
> `metric`: sklearn metric to optimize for <br>
> `datset_split`: random_state for datasetsplit <br>
> `name`: currently not user *optional* <br>

In [ ]:
all_results = []

for ds in all_datasets:
# Run search & print result
    results = cmp_hpt_methods(
        hpt_objs,
        SVC,
        (ds.data, ds.target),
        'accuracy', #http://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
        accuracy_score)
    all_results.append(results)

HTP using Baseline



HTP using BaysianSearch


In [ ]:
# ds_names = ['digits', 'iris', 'wine', 'breast ']
ds_names=['census']

In [ ]:
# Plot cross validation -e.g. tuning time
plot_by_ds(MEAN+CV_TIME, all_results, ds_names)
# Plot Test Accuracy
plot_by_ds(MEAN+TEST_ACC, all_results, ds_names)